<a href="https://colab.research.google.com/github/q890003/ML_logistic-regression-EM/blob/master/ML_hw4_EM_algorithm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import sys
import dill
import numpy as np
import math
from itertools import product 
from struct import unpack 
import numba as nb

def get_img(img_file_path):
    with open(img_file_path, "rb") as img_file:
        magic, num_imgs, rows, cols = unpack(">IIII", img_file.read(16))

        imgs_mtx = np.zeros((num_imgs, rows, cols), dtype=int)
        for img in range(num_imgs):
            for (row, col) in product(range(rows), range(cols)):
                tmp = unpack(">B", img_file.read(1))
                tmp = int(str(tmp[0]))
                imgs_mtx[img][row,col] = tmp
    return imgs_mtx, num_imgs

def get_label(lbl_file_path):
    with open(lbl_file_path, "rb") as label_file:
        magic, num_itms = unpack(">II", label_file.read(8))
        lbls_mtx = np.zeros((num_itms), dtype=int)
        for img in range(num_itms):
            label = unpack(">B", label_file.read(1))
            label = int(str(label[0]))
            lbls_mtx[img] = label
    return lbls_mtx

label_types = 10
gray_bins = 2
rows = 28
cols = 28

train_img_pth = "/content/drive/My Drive/Colab Notebooks/ML_hw4/train-images.idx3-ubyte"
train_label_pth = "/content/drive/My Drive/Colab Notebooks/ML_hw4/train-labels.idx1-ubyte"
train_img_mtx, train_num_imgs = get_img(train_img_pth)
train_label_mtx = get_label(train_label_pth)

labels_count = np.zeros((label_types), dtype=float)
for img in range(train_num_imgs):
    lbl = train_label_mtx[img]
    labels_count[lbl] += 1
    train_img_mtx[img] = train_img_mtx[img] // 128
  

In [29]:
@nb.jit
def estep(mu, train_img_mtx):
    for img in range(60000):
        for label in range(10):
            weight[img][label] = np.prod(2* (mu[label]*train_img_mtx[img] + (1- mu[label])*(1 - train_img_mtx[img]) ))

        weight[img] = weight[img]*lamda
        total = np.sum(weight[img])
        
        weight[img] /= total
    return weight

@nb.jit
def mstep(weight, train_img_mtx):
    mu_new = np.zeros((10,28,28), dtype=float)
    for label in range(10):
        for img in range(60000):
            mu_new[label] += weight[img][label] * train_img_mtx[img]  

    sigma_w = np.sum(weight, axis=0) 
    
    for label in range(10): 
        mu_new[label] /= sigma_w[label] 
    #recalculate lamda of each cluster. 
    lamda = sigma_w/train_num_imgs
    return mu_new, lamda

#For record
lamda_1 = np.empty(label_types)
mu_1 = np.empty((label_types,28,28))

lamda_9 = np.empty((label_types))
mu_9 = np.empty((label_types,28,28))

# init
lamda = np.full((label_types), 1/10)  #prob of a nmber under numbers 0~9
mu = np.random.rand(label_types,28,28)  #each cluster has 28*28 dim mu.
weight = np.zeros((train_num_imgs, label_types), dtype=float)  #record each cluster's weight to each data.
converge_count = 0
for it in range(10):
    # Expectation step:
    weight = estep(mu, train_img_mtx)
    # for img in range(train_num_imgs):
    #     for label in range(label_types):
    #         weight[img][label] = np.prod(2* (mu[label]*train_img_mtx[img] + (1- mu[label])*(1 - train_img_mtx[img]) ))

    #     weight[img] = weight[img]*lamda
    #     total = np.sum(weight[img])
        
    #     weight[img] /= total
 
    # Maximization step:
    # According to each cluster, recalculate mu of a cluster. 
    mu_new, lamda = mstep(weight, train_img_mtx)
    
    if it == 1:
        mu_1 = mu_new.copy()
        lamda_1 = lamda.copy()
    if it == 9:
        mu_9 = mu_new.copy()
        lamda_9 = lamda.copy()

    # if np.sum(abs(mu_new - mu)) < 0.001 :
    #     converge_count = it
    #     break
    converge_count = it

    #update mu
    mu = mu_new
    

<ipython-input-28-90f66f0ba326>:30: NumbaWarning: 
Compilation is falling back to object mode WITH looplifting enabled because Function "estep" failed type inference due to: Invalid use of Function(<built-in function setitem>) with argument(s) of type(s): (readonly array(float64, 2d, C), UniTuple(int64 x 2), float64)
 * parameterized
In definition 0:
    All templates rejected with literals.
In definition 1:
    All templates rejected without literals.
In definition 2:
    All templates rejected with literals.
In definition 3:
    All templates rejected without literals.
In definition 4:
    All templates rejected with literals.
In definition 5:
    All templates rejected without literals.
In definition 6:
    All templates rejected with literals.
In definition 7:
    All templates rejected without literals.
In definition 8:
    TypeError: Cannot modify value of type readonly array(float64, 2d, C)
    raised from /usr/local/lib/python3.6/dist-packages/numba/typing/arraydecl.py:179
In d

KeyboardInterrupt: ignored

In [23]:
imagination_number = mu_new.copy()
mapping = [5, 4, 0, 8, 6, 2, 1, 9, 7, 3]
imagination_number[imagination_number > 0.5] = 1
imagination_number[imagination_number <= 0.5] = 0
for label in range(label_types):
    print("labeled class ",label,":")
    for (r,c) in product(range(28), range(28)):
        print(int(imagination_number[mapping[label]][r][c]),end=" ")
        if c == 27:
            print("")
print("------------------------------------------------------------")

CM = np.zeros((10,4), dtype = int)   #TP, FN, FP, TN
for label in range(label_types):
    
    for img in range(train_num_imgs):
        predic_true = 1.0
        predic_false = 1.0
        true_label = train_label_mtx[img]
        predic_true *= np.sum( (  mu[label]   *train_img_mtx[img] + (1- mu[label])*(1 - train_img_mtx[img])) )
        predic_false *= np.sum(((1- mu[label])*train_img_mtx[img] +      mu[label]*(1 - train_img_mtx[img])) )

        if predic_true > 0.5:
            if true_label == label:
                CM[label][0] += 1
            else:
                CM[label][2] += 1
        else: 
            if true_label == label:
                CM[label][1] += 1
            else:
                CM[label][3] += 1
    
err = 0
for k in range(10):
    err += CM[k][1]
    Sensitivity = CM[k][0]/ (CM[k][0]+CM[k][1])
    Specificity = CM[k][3]/ (CM[k][2]+CM[k][3])
    print("")
    print("Confusion Matrix",k,":")
    print("\t\t","Predict number",k,"\tPredict not number",k)
    print("Is number",k,"\t\t",CM[k][0],"\t\t\t",CM[k][1])
    print("Isn't number",k,"\t\t",CM[k][2],"\t\t\t",CM[k][3])
    print("")
    print("Sensitivity (Successfully predict number ",k,")     :",Sensitivity)
    print("Specificity (Successfully predict not number ",k,") :",Specificity)
    print("")
    print("-----------------------------------------------------------")
print("Total iteration to converge: ",converge_count)
print("Total error rate: ",err / 60000)

labeled class  0 :
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 1 1 1 1 1 0 0 0 0 0 1 1 1 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 1 1 1 1 0 0 0 0 0 0 0 1 1 1 0 0 0 0 0 
0 0 0 0 0 0 0 0 1 1 1 1 0 0 0 0 0 0 0 0 1 1 1 0 0 0 0 0 
0 0 0 0 0 0 0 1 1 1 1 0 0 0 0 0 0 0 0 0 1 1 1 0 0 0 0 0 
0 0 0 0 0 0 0 1 1 1 0 0 0 0 0 0 0 0 0 0 1 1 1 0 0 0 0 0 
0 0 0 0 0 0 1 1 1 1 0 0 0 0 0 0 0 0 0 0 1 1 1 0 0 0 0 0 
0 0 0 0 0 0 1 1 1 0 0 0 0 0 0 0 0 0 0 0 1 1 1 0 0 0 0 0 
0 0 0 0 0 0 1 1 1 0 0 0 0 0 0 0 0 0 0 0 1 1 1 0 0 0 0 0 
0 0 0 0 0 0 